In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9715512528138228877
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22385000448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15781667858747911575
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


In [1]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose, UpSampling2D, Normalization, LeakyReLU
import numpy as np
import os
import matplotlib.pyplot as plt
from splitImage import loadImages as loadImage
from sklearn.cluster import KMeans

In [2]:
ColorMapping = {}

path = ".\\cityscapes_data\\cityscapes_data\\train"

all_ = os.listdir(path)
HIGH = [path+"\\"+i for i in all_]
# print(HIGH)
a = loadImage()
a.set(all_[:1000])
a.load(path)
ap = a.split()


160by160px
Split Data Count :  1000


In [3]:
x = np.array(ap)[:,0]
y = np.array(ap)[:,1]/10
yShape = y.shape
kmeans = KMeans(n_clusters=10)
kmeans.fit(y.reshape(-1,3))
# 클러스터 중심값
cluster_centers = kmeans.cluster_centers_

# 클러스터 할당
labels = kmeans.labels_

# 클러스터 중심값으로 통일시킨 데이터
data_uniform = np.sum(cluster_centers[labels], axis= 1)
index = -1
LableData = []
DataShape = len(data_uniform)
print(data_uniform)
print(np.unique(data_uniform, axis=0))
for i in (np.unique(data_uniform, axis=0)):
    print(i)
    if i in ColorMapping:
        index = ColorMapping[i]
    else:
        index += 1
        ColorMapping[i] = index
    # LableData.append(index)
# y = np.array(LableData).reshape(DataShape, 1)``

c:\Users\user\anaconda3\envs\tensor\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[ 1.35576256  1.35576256 20.88045542 ...  1.35576256  1.35576256
  1.35576256]
[ 1.35576256 14.29647203 16.11871308 20.88045542 28.91801299 29.07365864
 31.9066061  37.37814348 49.99652836 51.94508745]
1.3557625567011442
14.296472030268262
16.118713080316095
20.880455424130638
28.918012991844037
29.073658642749944
31.906606096502312
37.37814348373149
49.9965283622329
51.94508745179304


In [4]:
LableDatas = {}
for i,v in enumerate(np.unique(data_uniform, axis=0)):
    LableDatas[str(v)] = i
print(len(LableDatas))

10


In [5]:
print(DataShape)
print(data_uniform.shape)
mans = np.zeros(DataShape)
for i in range(DataShape):
    mans[i] = LableDatas[str(data_uniform[i])]
n,w,h,c = x.shape
y = mans.reshape(n,w,h,1)

25600000
(25600000,)


In [8]:
print(x.shape)
print(y.shape)

(1000, 160, 160, 3)
(1000, 160, 160, 1)


In [6]:
def unet():
    inputs = Input((160, 160, 3))
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = Normalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = LeakyReLU()(pool1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same', activation='relu')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = UpSampling2D(size=(2, 2))(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = UpSampling2D(size=(2, 2))(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    # conv10 = Conv2D(1, 1, activation='relu')(conv9)
    conv10 = Conv2D(1, 1, activation='softmax')(conv9)
    
    model = Model(inputs=[inputs], outputs=[conv10])

    return model

model = unet()
model.compile(tf.optimizers.RMSprop(0.0001, clipvalue=1.0), loss= tf.losses.CategoricalCrossentropy(),metrics=["accuracy"])
# validation accuracy, validation loss
history = model.fit(np.array(x),np.array(y),5, 500)    
# with tf.device("GPU:0"):
#     history = model.fit(np.array(x),np.array(y),55, 500)
model.save("kiki.h5")

Epoch 1/500


: 

: 